# RePlay recommender models comparison

We will show the main RePlay functionality and compare performance of RePlay models on well-known MovieLens dataset. If you have not used RePlay before, start with 01_replay_basics.ipynb which introduces base concepts and describe main classes and functionality.

### Dataset
We will compare RePlay models on __MovieLens 1m__. 

### Dataset preprocessing: 
Ratings greater than or equal to 3 are considered as positive interactions.

### Data split
Dataset is split by date so that 20% of the last interactions as are placed in the test part. Cold items and users are dropped.

### Predict:
We will predict top-10 most relevant films for each user.

### Metrics
Quality metrics used:__ndcg@k, hitrate@k, map@k, mrr@k__ for k = 1, 5, 10
Additional metrics used: __coverage@k__ and __surprisal@k__.

In [1]:
! pip install replay-rec[torch]==0.18.0rc0

  Using cached replay_rec-0.18.0rc0-py3-none-any.whl.metadata (11 kB)
  Using cached d3rlpy-2.7.0-py3-none-any.whl.metadata (11 kB)
  Using cached fixed_install_nmslib-2.1.2-cp310-cp310-linux_x86_64.whl
  Using cached gym-0.26.2-py3-none-any.whl
  Using cached hnswlib-0.7.0-cp310-cp310-linux_x86_64.whl
  Using cached implicit-0.7.2-cp310-cp310-manylinux2014_x86_64.whl.metadata (6.1 kB)
  Using cached lightautoml-0.3.8.1-py3-none-any.whl.metadata (16 kB)
  Using cached lightfm-1.17-cp310-cp310-linux_x86_64.whl
  Using cached llvmlite-0.43.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.8 kB)
  Using cached numba-0.60.0-cp310-cp310-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (2.7 kB)
  Using cached numpy-2.1.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
  Using cached optuna-3.2.0-py3-none-any.whl.metadata (17 kB)
  Using cached pandas-2.2.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (19 kB)
  U

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
%config Completer.use_jedi = False

In [2]:
import warnings
from optuna.exceptions import ExperimentalWarning
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=ExperimentalWarning)

/home/lockr/projects/Sber_RecSys-hack/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
import logging
import time


from replay.data import Dataset, FeatureHint, FeatureInfo, FeatureSchema, FeatureType
from replay.data.dataset_utils import DatasetLabelEncoder
from replay.metrics import Coverage, HitRate, MRR, MAP, NDCG, Surprisal, Experiment, OfflineMetrics
from replay.models import (
    ALSWrap, 
    ItemKNN,
    SLIM, 
    PopRec,
    RandomRec,
    UCB,
    Wilson, 
    Word2VecRec,
)

from replay.utils.session_handler import State
from replay.splitters import LastNSplitter
from replay.utils.spark_utils import convert2spark, get_log_info

In [5]:
logger = logging.getLogger("replay")

In [6]:
K = 10
K_list_metrics = [1, 5, 10]
BUDGET = 5
SEED = 42

## 0. Preprocessing <a name='data-preparator'></a>

### 0.1 Data loading

In [7]:
import pandas as pd
import numpy as np
data = pd.read_parquet("train_smm.parquet")

### 0.2. Dataset preparation

#### interactions preprocessing

- converting to spark dataframe
- separating positive and negative feedback interactions
- spliting data for train/test and optimization dataframes
- creating Dataset instances for experiments
- encoding datasets

In [8]:
interactions = data
interactions.describe()

,user_id,item_id,timestamp,rating
count,2.068959e+07,2.068959e+07,2.068959e+07,2.068959e+07
mean,5.511610e+06,8.349397e+04,1.678635e+18,3.304713e+00
std,2.596605e+06,4.844011e+04,2.407096e+15,1.375845e+01
min,1.000010e+06,0.000000e+00,1.673741e+18,1.554808e+00
25%,3.269195e+06,4.165800e+04,1.676758e+18,1.554808e+00
50%,5.513310e+06,8.324200e+04,1.678792e+18,1.554808e+00
75%,7.770887e+06,1.256090e+05,1.680779e+18,2.802427e+00
max,9.999982e+06,1.677970e+05,1.682414e+18,1.964221e+04


In [9]:
# will consider ratings >= 3 as positive feedback. A positive feedback is treated with rating = 1
only_positives_interactions = interactions[interactions.rating >= 3]
only_positives_interactions.loc['rating'] = 1

/tmp/ipykernel_11181/1361020441.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  only_positives_interactions.loc['rating'] = 1


In [10]:
# train/test split 
train_spl = LastNSplitter(
    N=K,
    divide_column="user_id",
    drop_cold_items=True,
    drop_cold_users=True,
    query_column="user_id",
    item_column="item_id",
)
train, test = train_spl.split(only_positives_interactions)

In [11]:
# train/test split for hyperparameters selection
opt_train, opt_val = train_spl.split(train)
opt_train.count(), opt_val.count()

(user_id      1068352
 item_id      1068352
 timestamp    1068352
 rating       1068352
 dtype: int64,
 user_id      467901
 item_id      467901
 timestamp    467901
 rating       467901
 dtype: int64)

In [12]:
# negative feedback will be used for Wilson and UCB models
only_negatives_log = interactions[interactions.rating < 3]
only_negatives_log['rating'] = 0


/tmp/ipykernel_11181/467379785.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  only_negatives_log['rating'] = 0


In [13]:
feature_schema = FeatureSchema(
    [
        FeatureInfo(
            column="user_id",
            feature_type=FeatureType.CATEGORICAL,
            feature_hint=FeatureHint.QUERY_ID,
        ),
        FeatureInfo(
            column="item_id",
            feature_type=FeatureType.CATEGORICAL,
            feature_hint=FeatureHint.ITEM_ID,
        ),
        FeatureInfo(
            column="rating",
            feature_type=FeatureType.NUMERICAL,
            feature_hint=FeatureHint.RATING,
        ),
        FeatureInfo(
            column="timestamp",
            feature_type=FeatureType.NUMERICAL,
            feature_hint=FeatureHint.TIMESTAMP,
        ),
    ]
)

In [14]:
all_dataset = Dataset(
    feature_schema=feature_schema,
    interactions=interactions,
)

train_dataset = Dataset(
    feature_schema=feature_schema,
    interactions=train,
)

test_dataset = Dataset(
    feature_schema=feature_schema,
    interactions=test,
)

opt_train_dataset = Dataset(
    feature_schema=feature_schema,
    interactions=opt_train,
)

opt_val_dataset = Dataset(
    feature_schema=feature_schema,
    interactions=opt_val,
)

In [15]:
encoder = DatasetLabelEncoder()
encoder.fit(all_dataset)
train_dataset = encoder.transform(train_dataset)
test_dataset = encoder.transform(test_dataset)
opt_train_dataset = encoder.transform(opt_train_dataset)
opt_val_dataset = encoder.transform(opt_val_dataset)

# 1. Metrics definition

In [16]:
# experiment is used for metrics calculation
e = Experiment(
    [
        MAP(K), 
        NDCG(K), 
        HitRate(K_list_metrics), 
        Coverage(K),
        Surprisal(K),
        MRR(K),
    ],
    test_dataset.interactions,
    train_dataset.interactions,
    query_column=train_dataset.feature_schema.query_id_column,
    item_column=train_dataset.feature_schema.item_id_column,
    rating_column=train_dataset.feature_schema.interactions_rating_column,
)

In [17]:
from replay.experimental.models.mult_vae import MultVAE

ImportError: cannot import name 'cosine_similarity' from 'replay.utils.spark_utils' (/home/lockr/projects/Sber_RecSys-hack/.venv/lib/python3.10/site-packages/replay/utils/spark_utils.py)

# 2. Models training

In [71]:
def fit_predict_add_res(name, model, experiment, train_dataset, suffix=''):
    """
    Run fit_predict for the `model`, measure time on fit_predict and evaluate metrics
    """
    start_time=time.time()
    
    logs = {'dataset': train_dataset}
    predict_params = {'k': K, 'queries': test_dataset.query_ids}

    predict_params.update(logs)

    model.fit(**logs)
    fit_time = time.time() - start_time

    pred=model.predict(**predict_params)
    pred.cache()
    pred.count()
    predict_time = time.time() - start_time - fit_time

    experiment.add_result(name + suffix, pred)
    metric_time = time.time() - start_time - fit_time - predict_time
    experiment.results.loc[name + suffix, 'fit_time'] = fit_time
    experiment.results.loc[name + suffix, 'predict_time'] = predict_time
    experiment.results.loc[name + suffix, 'metric_time'] = metric_time
    experiment.results.loc[name + suffix, 'full_time'] = (fit_time + 
                                                          predict_time +
                                                          metric_time)
    pred.unpersist()
    print(experiment.results[['NDCG@{}'.format(K), 'MRR@{}'.format(K), 'Coverage@{}'.format(K), 'fit_time']].sort_values('NDCG@{}'.format(K), ascending=False))

In [72]:
def full_pipeline(models, experiment, train_dataset, suffix='', budget=BUDGET):
    """
    For each model:
        -  if required: run hyperparameters search, set best params and save param values to `experiment`
        - pass model to `fit_predict_add_res`        
    """
    
    for name, [model, params] in models.items():
        model.logger.info(msg='{} started'.format(name))
        if params != 'no_opt':
            model.logger.info(msg='{} optimization started'.format(name))
            best_params = model.optimize(opt_train_dataset, 
                                         opt_val_dataset, 
                                         param_borders=params, 
                                         k=K, 
                                         budget=budget)
            logger.info(msg='best params for {} are: {}'.format(name, best_params))
            model.set_params(**best_params)
        
        logger.info(msg='{} fit_predict started'.format(name))
        fit_predict_add_res(name, model, experiment, train_dataset, suffix)
        # here we call protected attribute to get all parameters set during model initialization
        experiment.results.loc[name + suffix, 'params'] = str(model._init_args)

## 2.1. Non-personalized models

In [73]:
non_personalized_models = {
    'Popular': [PopRec(), 'no_opt'], 
    'Random (uniform)': [RandomRec(seed=SEED, distribution='uniform'), 'no_opt'], 
    'Random (popularity-based)': [RandomRec(seed=SEED, distribution='popular_based'), {"alpha": [-0.5, 100]}]
}

In [74]:
%%time
full_pipeline(non_personalized_models, e, train_dataset)

AttributeError: 'DataFrame' object has no attribute 'select'

In [ ]:
e.results.sort_values('NDCG@10', ascending=False)

,MAP@10,NDCG@10,HitRate@1,HitRate@5,HitRate@10,Coverage@10,Surprisal@10,MRR@10,fit_time,predict_time,metric_time,full_time,params
Popular,0.157302,0.243711,0.284460,0.530290,0.645303,0.033903,0.118354,0.390426,3.411652,8.815796,8.258563,20.486010,"{'use_rating': False, 'add_cold_items': True, ..."
Wilson,0.045002,0.092121,0.083406,0.345040,0.414399,0.017092,0.262190,0.180976,3.453442,6.875320,6.423975,16.752737,"{'alpha': 0.05, 'add_cold_items': True, 'cold_..."
Random (popularity-based),0.031447,0.074118,0.079017,0.269535,0.399473,0.635192,0.314159,0.160990,3.927062,6.529720,6.674805,17.131586,"{'distribution': 'popular_based', 'alpha': 0.0..."
Random (uniform),0.009661,0.026336,0.032485,0.099210,0.181738,0.954609,0.537563,0.065413,3.629176,8.216730,7.108498,18.954404,"{'distribution': 'uniform', 'alpha': 0.0, 'see..."
UCB,0.000124,0.000458,0.000000,0.000878,0.004390,0.003082,1.000000,0.001024,3.415656,7.359273,5.900311,16.675241,"{'exploration_coef': 0.5, 'sample': False, 'se..."


UCB is developed for iterative learning and improve its quality over time as the exploration reduces.

In [ ]:

e.results.to_csv('res_21_rel_1.csv')

## 2.2  Personalized models without features

In [ ]:
common_models = {
  'Implicit ALS': [ALSWrap(seed=SEED), None], 
  'Explicit ALS': [ALSWrap(seed=SEED, implicit_prefs=False), None], 
  'ItemKNN': [ItemKNN(), None], 
  'SLIM': [SLIM(seed=SEED), None],
  'Word2Vec': [Word2VecRec(seed=SEED), None],
}

In [ ]:
%%time
full_pipeline(common_models, e, train_dataset)

10-Nov-23 17:26:51, replay, INFO: Implicit ALS started
10-Nov-23 17:26:51, replay, INFO: Implicit ALS optimization started
[I 2023-11-10 17:26:51,161] A new study created in memory with name: no-name-346690a7-8179-4a82-8179-edbfdb8bf4a1
/root/anaconda3/envs/replay/lib/python3.9/site-packages/pyspark/sql/context.py:125: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(
[I 2023-11-10 17:27:08,366] Trial 0 finished with value: 0.2106789106218419 and parameters: {'rank': 10}. Best is trial 0 with value: 0.2106789106218419.
[I 2023-11-10 17:27:24,184] Trial 1 finished with value: 0.1986859010576274 and parameters: {'rank': 26}. Best is trial 0 with value: 0.2106789106218419.
[I 2023-11-10 17:27:41,154] Trial 2 finished with value: 0.1834988563514584 and parameters: {'rank': 40}. Best is trial 0 with value: 0.2106789106218419.
[I 2023-11-10 17:28:51,126] Trial 3 finished with value: 0.16951885280473594 and parameters: {'rank': 168}. Best is 

                            NDCG@10    MRR@10  Coverage@10  fit_time
Implicit ALS               0.255083  0.412843     0.147380  4.430974
Popular                    0.243711  0.390426     0.033903  3.411652
Wilson                     0.092121  0.180976     0.017092  3.453442
Random (popularity-based)  0.074118  0.160990     0.635192  3.927062
Random (uniform)           0.026336  0.065413     0.954609  3.629176
UCB                        0.000458  0.001024     0.003082  3.415656


/root/anaconda3/envs/replay/lib/python3.9/site-packages/pyspark/sql/context.py:125: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(
[I 2023-11-10 17:29:43,046] Trial 0 finished with value: 0.01812132636739519 and parameters: {'rank': 10}. Best is trial 0 with value: 0.01812132636739519.
[I 2023-11-10 17:30:18,482] Trial 1 finished with value: 0.01975373348436346 and parameters: {'rank': 107}. Best is trial 1 with value: 0.01975373348436346.
[I 2023-11-10 17:30:36,546] Trial 2 finished with value: 0.02675199192979083 and parameters: {'rank': 46}. Best is trial 2 with value: 0.02675199192979083.
[I 2023-11-10 17:30:50,656] Trial 3 finished with value: 0.014121230585317253 and parameters: {'rank': 9}. Best is trial 2 with value: 0.02675199192979083.
[I 2023-11-10 17:31:24,553] Trial 4 finished with value: 0.014698300217981774 and parameters: {'rank': 102}. Best is trial 2 with value: 0.02675199192979083.
10-Nov-23 17:31:24, replay, INFO

                            NDCG@10    MRR@10  Coverage@10  fit_time
Implicit ALS               0.255083  0.412843     0.147380  4.430974
Popular                    0.243711  0.390426     0.033903  3.411652
Wilson                     0.092121  0.180976     0.017092  3.453442
Random (popularity-based)  0.074118  0.160990     0.635192  3.927062
Random (uniform)           0.026336  0.065413     0.954609  3.629176
Explicit ALS               0.019994  0.045540     0.431213  7.814560
UCB                        0.000458  0.001024     0.003082  3.415656


[I 2023-11-10 17:32:09,776] Trial 0 finished with value: 0.2081514555056189 and parameters: {'num_neighbours': 10, 'shrink': 0, 'weighting': None}. Best is trial 0 with value: 0.2081514555056189.
[I 2023-11-10 17:32:18,049] Trial 1 finished with value: 0.21742351926120368 and parameters: {'num_neighbours': 27, 'shrink': 4, 'weighting': 'bm25'}. Best is trial 1 with value: 0.21742351926120368.
[I 2023-11-10 17:32:26,907] Trial 2 finished with value: 0.22506007009967735 and parameters: {'num_neighbours': 84, 'shrink': 11, 'weighting': 'bm25'}. Best is trial 2 with value: 0.22506007009967735.
[I 2023-11-10 17:32:35,270] Trial 3 finished with value: 0.23090776162732826 and parameters: {'num_neighbours': 73, 'shrink': 51, 'weighting': None}. Best is trial 3 with value: 0.23090776162732826.
[I 2023-11-10 17:32:43,325] Trial 4 finished with value: 0.2285854011339672 and parameters: {'num_neighbours': 50, 'shrink': 66, 'weighting': None}. Best is trial 3 with value: 0.23090776162732826.
10-Nov

                            NDCG@10    MRR@10  Coverage@10   fit_time
ItemKNN                    0.257599  0.413689     0.058840  16.430850
Implicit ALS               0.255083  0.412843     0.147380   4.430974
Popular                    0.243711  0.390426     0.033903   3.411652
Wilson                     0.092121  0.180976     0.017092   3.453442
Random (popularity-based)  0.074118  0.160990     0.635192   3.927062
Random (uniform)           0.026336  0.065413     0.954609   3.629176
Explicit ALS               0.019994  0.045540     0.431213   7.814560
UCB                        0.000458  0.001024     0.003082   3.415656


/root/work/replay_gitlab/RePlay/replay/optimization/optuna_objective.py:77: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  res[param] = suggest_fn(param, low=low, high=high)
[I 2023-11-10 17:33:31,375] Trial 0 finished with value: 0.186897920331224 and parameters: {'beta': 0.01, 'lambda_': 0.01}. Best is trial 0 with value: 0.186897920331224.
[I 2023-11-10 17:33:51,374] Trial 1 finished with value: 0.18130428970784088 and parameters: {'beta': 0.0023960955966723456, 'lambda_': 6.117746940780402e-05}. Best is trial 0 with value: 0.186897920331224.
[I 2023-11-10 17:34:05,087] Trial 2 finished with value: 0.19167760516064122 and parameters: {'beta': 0.004819710134847662, 'lambda_': 0.03444850489984713}. Best is trial 2 with value: 0.19167760516064122.
[I 2023-11-10 17:34:25,735] Trial 3 finished with value: 0.17566010185645373 and p

                            NDCG@10    MRR@10  Coverage@10   fit_time
ItemKNN                    0.257599  0.413689     0.058840  16.430850
Implicit ALS               0.255083  0.412843     0.147380   4.430974
Popular                    0.243711  0.390426     0.033903   3.411652
SLIM                       0.237732  0.412639     0.085458   8.316423
Wilson                     0.092121  0.180976     0.017092   3.453442
Random (popularity-based)  0.074118  0.160990     0.635192   3.927062
Random (uniform)           0.026336  0.065413     0.954609   3.629176
Explicit ALS               0.019994  0.045540     0.431213   7.814560
UCB                        0.000458  0.001024     0.003082   3.415656


/root/anaconda3/envs/replay/lib/python3.9/site-packages/pyspark/sql/context.py:125: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(
[I 2023-11-10 17:35:37,137] Trial 0 finished with value: 0.1294242567130421 and parameters: {'rank': 100, 'window_size': 1, 'use_idf': False}. Best is trial 0 with value: 0.1294242567130421.
[I 2023-11-10 17:36:21,890] Trial 1 finished with value: 0.03256363722289059 and parameters: {'rank': 87, 'window_size': 36, 'use_idf': True}. Best is trial 0 with value: 0.1294242567130421.
[I 2023-11-10 17:37:20,820] Trial 2 finished with value: 0.03485106484524721 and parameters: {'rank': 196, 'window_size': 38, 'use_idf': True}. Best is trial 0 with value: 0.1294242567130421.
[I 2023-11-10 17:38:16,795] Trial 3 finished with value: 0.04023855538139732 and parameters: {'rank': 128, 'window_size': 55, 'use_idf': False}. Best is trial 0 with value: 0.1294242567130421.
[I 2023-11-10 17:39:27,691] Trial 4 finished wit

                            NDCG@10    MRR@10  Coverage@10   fit_time
ItemKNN                    0.257599  0.413689     0.058840  16.430850
Implicit ALS               0.255083  0.412843     0.147380   4.430974
Popular                    0.243711  0.390426     0.033903   3.411652
SLIM                       0.237732  0.412639     0.085458   8.316423
Word2Vec                   0.138505  0.251616     0.152984   4.772598
Wilson                     0.092121  0.180976     0.017092   3.453442
Random (popularity-based)  0.074118  0.160990     0.635192   3.927062
Random (uniform)           0.026336  0.065413     0.954609   3.629176
Explicit ALS               0.019994  0.045540     0.431213   7.814560
UCB                        0.000458  0.001024     0.003082   3.415656
CPU times: user 9.24 s, sys: 2.14 s, total: 11.4 s
Wall time: 13min 10s


In [ ]:
(
    e.results
    .sort_values('NDCG@10', ascending=False)
    [["Coverage@10", "HitRate@1", "HitRate@5", "HitRate@10", "MAP@10", "MRR@10", "NDCG@10", "Surprisal@10", "fit_time", "predict_time", "metric_time", "full_time"]]
)

,Coverage@10,HitRate@1,HitRate@5,HitRate@10,MAP@10,MRR@10,NDCG@10,Surprisal@10,fit_time,predict_time,metric_time,full_time
ItemKNN,0.058840,0.303775,0.555751,0.654083,0.167841,0.413689,0.257599,0.140874,16.430850,8.416453,6.230795,31.078098
Implicit ALS,0.147380,0.302019,0.566286,0.677788,0.162911,0.412843,0.255083,0.170293,4.430974,10.465904,5.813219,20.710098
Popular,0.033903,0.284460,0.530290,0.645303,0.157302,0.390426,0.243711,0.118354,3.411652,8.815796,8.258563,20.486010
SLIM,0.085458,0.298507,0.570676,0.669008,0.143271,0.412639,0.237732,0.154963,8.316423,5.008076,5.497450,18.821949
Word2Vec,0.152984,0.151010,0.385426,0.498683,0.073364,0.251616,0.138505,0.245530,4.772598,23.490503,5.852325,34.115426
Wilson,0.017092,0.083406,0.345040,0.414399,0.045002,0.180976,0.092121,0.262190,3.453442,6.875320,6.423975,16.752737
Random (popularity-based),0.635192,0.079017,0.269535,0.399473,0.031447,0.160990,0.074118,0.314159,3.927062,6.529720,6.674805,17.131586
Random (uniform),0.954609,0.032485,0.099210,0.181738,0.009661,0.065413,0.026336,0.537563,3.629176,8.216730,7.108498,18.954404
Explicit ALS,0.431213,0.010536,0.086918,0.168569,0.006395,0.045540,0.019994,0.591741,7.814560,10.279030,5.894241,23.987831
UCB,0.003082,0.000000,0.000878,0.004390,0.000124,0.001024,0.000458,1.000000,3.415656,7.359273,5.900311,16.675241


In [ ]:
e.results.to_csv('res_22_rel_1.csv')

# 3. Results

The best results by quality and time were shown by the commonly-used models such as ItemKNN, ALS, Popular and SLIM

In [ ]:
e.results.sort_values('NDCG@10', ascending=False).head(5)

,MAP@10,NDCG@10,HitRate@1,HitRate@5,HitRate@10,Coverage@10,Surprisal@10,MRR@10,fit_time,predict_time,metric_time,full_time,params
ItemKNN,0.167841,0.257599,0.303775,0.555751,0.654083,0.058840,0.140874,0.413689,16.430850,8.416453,6.230795,31.078098,"{'shrink': 51, 'use_rating': False, 'num_neigh..."
Implicit ALS,0.162911,0.255083,0.302019,0.566286,0.677788,0.147380,0.170293,0.412843,4.430974,10.465904,5.813219,20.710098,"{'rank': 10, 'implicit_prefs': True, 'seed': 42}"
Popular,0.157302,0.243711,0.284460,0.530290,0.645303,0.033903,0.118354,0.390426,3.411652,8.815796,8.258563,20.486010,"{'use_rating': False, 'add_cold_items': True, ..."
SLIM,0.143271,0.237732,0.298507,0.570676,0.669008,0.085458,0.154963,0.412639,8.316423,5.008076,5.497450,18.821949,"{'beta': 0.004819710134847662, 'lambda_': 0.03..."
Word2Vec,0.073364,0.138505,0.151010,0.385426,0.498683,0.152984,0.245530,0.251616,4.772598,23.490503,5.852325,34.115426,"{'rank': 100, 'window_size': 1, 'use_idf': Fal..."
